In [2]:
!pip install librosa


  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for audioread>=2.1.9 from https://files.pythonhosted.org/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for pooch>=1.0 from https://files.pythonhosted.org/packages/1a/a5/5174dac3957ac412e80a00f30b6507031fcab7000afc9ea0ac413bddcff2/pooch-1.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for soxr>=0.3.2 from https://files.pythonhosted.org/packages/2b/97/cbce72f9c8b5c9c667eb55dc55be20a87c610dba55c0466c77498c1a8c97/soxr-0.3.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/

In [2]:
!pip install scikit-maad

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 1.4 MB/s eta 0:00:001.5 MB/s eta 0:00:01


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
from pathlib import Path
import sys
import os
import time
import warnings
# suppress all warnings
warnings.filterwarnings("ignore")

from scipy import signal
import librosa
import librosa.display
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from maad import sound, util, rois

In [2]:
def grab_audio(path, audio_format='mp3'):
    filelist = []
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if name[-3:].casefold() == audio_format and name[:2] != '._':
                filelist.append(os.path.join(root, name))
    return filelist

In [3]:
XC_ROOTDIR = '/home/aarav/Desktop/BIrd dataset'

In [4]:
XC_DIR = 'chara_dataset'

# data = [['Asian Koel', 'Eudynamys scolopaceus'],
# ['Large-billed Crow', 'Corvus macrorhynchos'],
# ['Black Kite', 'Milvus migrans'],
# ['Long-tailed Duck', 'Clangula hyemalis'],
# ['Bristled grassbird', 'Chaetornis striata'],
# ['Red-billed Blue Magpie', 'Urocissa erythroryncha'],
# ['Rose-ringed Parakeet', 'Psittacula krameri'],
# ['Common Cuckoo', 'Cuculus canorus'],
# ["Common Tailorbird", "Orthotomus sutorius"],
# ['Common Pochard', 'Aythya ferina'],
# ['Daurian Redstart', 'Phoenicurus auroreus'],
# ['Rufous Treepie', 'Dendrocitta vagabunda'],
# ['Barn Swallow', 'Hirundo rustica'],
# ['Mountain Scops Owl', 'Otus spilocephalus'],
# ['Common Myna', 'Acridotheres tristis'],
# ['Common Wood Pigeon', 'Columba palumbus'],
# ['Great Barbet', 'Psilopogon virens'],
# ['Collared Owlet ', 'Taenioptynx brodiei'],
# ['Golden-throated Barbet', 'Psilopogon franklinii'],
# ['Grey-chinned Minivet', 'Pericrocotus solaris'],
# ['Striated Bulbul ', 'Alcurus striatus'],
# ['Rustic Bunting', 'Emberiza rustica'],
# ['Saker Falcon', 'Falco cherrug'],
# ['Changeable hawk-eagle', 'Nisaetus cirrhatus'],
# ['Sarus Crane', 'Antigone antigone'],
# ['Greater Spotted Eagle', 'Clanga clanga'],
# ['Spotted Owlet', 'Athene brama'],
# ['Great Slaty Woodpecker', 'Mulleripicus pulverulentus'],
# ['Slender-billed Babbler', 'Turdoides longirostris'],
# ['Spotted Dove', 'Spilopelia chinensis'],
# ['Grey Treepie', 'Dendrocitta formosae'],
# ['Himalayan Monal', 'Lophophorus impejanus'],
# ['Swamp Francolin', 'Francolinus gularis'],
# ['House Crow', 'Corvus splendens'],
# ['Swamp Grass-babbler', 'Graminicola bengalensis'],
# ['House Sparrow', 'Passer domesticus'],
# ["Black Drongo", "Dicrurus macrocercus"]
# ]

data = [['White-throated Kingfisher', 'Halcyon smyrnensis'],
['Indian Peafowl', 'Pavo cristatus'],
['Common Quail', 'Coturnix coturnix'],
['Red Junglefowl', 'Gallus gallus'],
['Little Grebe', 'Tachybaptus ruficollis'],
['Great Crested Grebe', 'Podiceps cristatus'],
['Rock Dove', 'Columba livia'],
['Laughing Dove', 'Spilopelia senegalensis'],
['Greater Coucal', 'Centropus sinensis'],
['Plaintive Cuckoo', 'Cacomantis merulinus'],
['Large Hawk-Cuckoo', 'Hierococcyx sparverioides'],
['Indian Cuckoo', 'Cuculus micropterus'],
['Large-tailed Nightjar', 'Caprimulgus macrurus'],
['Common Swift', 'Apus apus'],
['Water Rail', 'Rallus aquaticus'],
['Spotted Crake', 'Porzana porzana'],
['Common Moorhen', 'Gallinula chloropus'],
['Eurasian Coot', 'Fulica atra'],
['White-breasted Waterhen', 'Amaurornis phoenicurus'],
["Baillon's Crake", 'Zapornia pusilla'],
['Common Crane', 'Grus grus'],
['Black-winged Kite', 'Elanus caeruleus'],
['Western Marsh Harrier', 'Circus aeruginosus']

]

In [5]:
df_species = pd.DataFrame(data,columns =['english name',
                                         'scientific name'])
gen = []
sp = []
for name in df_species['scientific name']:
    gen.append(name.rpartition(' ')[0])
    sp.append(name.rpartition(' ')[2])

In [6]:
df_query = pd.DataFrame()
df_query['param1'] = gen
df_query['param2'] = sp
# df_query['param5 ='len:"5-120"'
df_query['param3'] ='q:">C"'

# Get recordings metadata corresponding to the query
df_dataset= util.xc_multi_query(df_query,
                                 format_time = False,
                                 format_date = False,
                                 verbose=True)

Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Halcyon%20smyrnensis%20q:">C"&page=1
Found 1 pages in total.
Saved metadata for 175 files
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Pavo%20cristatus%20q:">C"&page=1
Found 1 pages in total.
Saved metadata for 84 files
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Coturnix%20coturnix%20q:">C"&page=1
Loading page 2...
https://www.xeno-canto.org/api/2/recordings?query=Coturnix%20coturnix%20q:">C"&page=2
Found 2 pages in total.
Saved metadata for 849 files
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Gallus%20gallus%20q:">C"&page=1
Found 1 pages in total.
Saved metadata for 130 files
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Tachybaptus%20ruficollis%20q:">C"&page=1
Loading page 2...
https://www.xeno-canto.org/api/2/recordings?query=Tachybaptus%20ruficollis%20q:">C"&page=2
Found 2 pages in total.
Saved metadata for 921 files
Load

In [7]:
df_dataset

,id,gen,sp,ssp,group,en,rec,cnt,loc,lat,...,rmk,bird-seen,animal-seen,playback-used,temp,regnr,auto,dvc,mic,smp
0,845720,Halcyon,smyrnensis,fokiensis,birds,White-throated Kingfisher,Geoff Carey,China,"Nam Chung, Tai Po, New Territories, Hong Kong",22.5178,...,Inactive fish ponds and scattered shrubland. 8...,yes,yes,no,,,no,Sound Devices 722,Telinga Twin Science,48000
1,845719,Halcyon,smyrnensis,fokiensis,birds,White-throated Kingfisher,Geoff Carey,China,"Mai Po NR, New Territories, Hong Kong",22.4896,...,"In flight at wetland nature reserve. 8dB NR, 1...",yes,yes,no,,,no,Mix-Pre 3 (ii),Telinga Twin Science,48000
2,845718,Halcyon,smyrnensis,fokiensis,birds,White-throated Kingfisher,Geoff Carey,China,"Luo Shuai Village, Yinggeling, Hainan",19.1891,...,Perched on wire overlooking wooded riverine va...,yes,yes,no,,,no,Fostex FR2-LE,Wildtronics Mono,48000
3,833597,Halcyon,smyrnensis,perpulchra,birds,White-throated Kingfisher,Albert Noorlander,Thailand,"Bang Kacha, Mueang Chanthaburi District, Chant...",12.5564,...,,yes,yes,no,,,no,H4N Pro,Rode NTG 2,44100
4,809703,Halcyon,smyrnensis,,birds,White-throated Kingfisher,Marc Anderson,Nepal,"Bardiya National Park, Bardiya, Lumbini Province",28.4625,...,,no,no,no,,,unknown,,,44100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10254,104563,Circus,aeruginosus,,birds,Western Marsh Harrier,Marc,Bulgaria,Durankulak,43.6880,...,"Call from a male bird, hanging in the air at 2...",unknown,unknown,unknown,,,no,,,44100
10255,100623,Circus,aeruginosus,aeruginosus,birds,Western Marsh Harrier,Jarek Matusiak,Poland,Janowica village,51.3098,...,song of a lonely male -in the 'butterfly' - li...,unknown,unknown,unknown,,,no,,,48000
10256,100620,Circus,aeruginosus,aeruginosus,birds,Western Marsh Harrier,Jarek Matusiak,Poland,Meadows near Janówek near Narew River (Łąki ko...,52.4540,...,call female on the nesting platform begging fo...,unknown,unknown,unknown,,,no,,,48000
10257,81916,Circus,aeruginosus,,birds,Western Marsh Harrier,Matthias Feuersenger,Germany,"Kirschgartshausen, Mannheim",49.5850,...,Advertising Male flying far overhead.Recording...,yes,yes,no,,,no,,,44100


In [10]:
df_dataset_download = util.xc_selection(df_dataset,
                               max_nb_files=70,
                               max_length='03:00',
                               min_length='00:10',
                               min_quality='B',
                               verbose = True )

Halcyon smyrnensis
    ... request 70 files of quality A
    --> found 22 files of quality A and 00:10<length<03:00
    ... request 48 files of quality B
    --> found 48 files of quality B and 00:10<length<03:00
    total files : 70
-----------------------------------------
Pavo cristatus
    ... request 70 files of quality A
    --> found 24 files of quality A and 00:10<length<03:00
    ... request 46 files of quality B
    --> found 22 files of quality B and 00:10<length<03:00
    total files : 46
-----------------------------------------
Coturnix coturnix
    ... request 70 files of quality A
    --> found 70 files of quality A and 00:10<length<03:00
    total files : 70
-----------------------------------------
Gallus gallus
    ... request 70 files of quality A
    --> found 65 files of quality A and 00:10<length<03:00
    ... request  5 files of quality B
    --> found  5 files of quality B and 00:10<length<03:00
    total files : 70
-----------------------------------------
Tac

In [11]:
util.xc_download(df_dataset_download,
                 rootdir = XC_ROOTDIR,
                 dataset_name= XC_DIR,
                 overwrite=False,
                 save_csv= True,
                 verbose = True)

A total of 1586 files will be downloaded
Creating subdirectory /home/aarav/Desktop/BIrd dataset/chara_dataset/Halcyon smyrnensis_White-throated Kingfisher for downloaded files...
Saving file  1 / 1586 : https://xeno-canto.org/845718/download
Saving file  2 / 1586 : https://xeno-canto.org/797016/download
Saving file  3 / 1586 : https://xeno-canto.org/845720/download
Saving file  4 / 1586 : https://xeno-canto.org/507756/download
Saving file  5 / 1586 : https://xeno-canto.org/809702/download
Saving file  6 / 1586 : https://xeno-canto.org/365385/download
Saving file  7 / 1586 : https://xeno-canto.org/742045/download
Saving file  8 / 1586 : https://xeno-canto.org/547540/download
Saving file  9 / 1586 : https://xeno-canto.org/778390/download
Saving file  10 / 1586 : https://xeno-canto.org/472751/download
Saving file  11 / 1586 : https://xeno-canto.org/620793/download
Saving file  12 / 1586 : https://xeno-canto.org/797018/download
Saving file  13 / 1586 : https://xeno-canto.org/311306/downloa

,gen,sp,ssp,group,en,rec,cnt,loc,lat,lng,...,bird-seen,animal-seen,playback-used,temp,regnr,auto,dvc,mic,smp,fullfilename
id,,,,,,,,,,,,,,,,,,,,,
845718,Halcyon,smyrnensis,fokiensis,birds,White-throated Kingfisher,Geoff Carey,China,"Luo Shuai Village, Yinggeling, Hainan",19.1891,109.6098,...,yes,yes,no,,,no,Fostex FR2-LE,Wildtronics Mono,48000,/home/aarav/Desktop/BIrd dataset/chara_dataset...
797016,Halcyon,smyrnensis,,birds,White-throated Kingfisher,JISHNU KIZHAKKILLAM,India,Porora,11.9632,75.5785,...,yes,yes,yes,,,no,OLYMPUS LS-12,RODE NTG4+,48000,/home/aarav/Desktop/BIrd dataset/chara_dataset...
845720,Halcyon,smyrnensis,fokiensis,birds,White-throated Kingfisher,Geoff Carey,China,"Nam Chung, Tai Po, New Territories, Hong Kong",22.5178,114.2090,...,yes,yes,no,,,no,Sound Devices 722,Telinga Twin Science,48000,/home/aarav/Desktop/BIrd dataset/chara_dataset...
507756,Halcyon,smyrnensis,,birds,White-throated Kingfisher,Jean Roché,India,"Corbett NP, Uttaranchal",29.5334,78.9334,...,unknown,unknown,no,,,no,,,44100,/home/aarav/Desktop/BIrd dataset/chara_dataset...
809702,Halcyon,smyrnensis,,birds,White-throated Kingfisher,Marc Anderson,Nepal,"Bardiya National Park, Bardiya, Lumbini Province",28.4625,81.2348,...,no,no,no,,,unknown,,,44100,/home/aarav/Desktop/BIrd dataset/chara_dataset...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608629,Circus,aeruginosus,aeruginosus,birds,Western Marsh Harrier,Jarek Matusiak,Hungary,"Kiskunhalas, Kiskunhalas District, Bács-Kiskun",46.4362,19.4842,...,unknown,unknown,unknown,,,no,,,44100,/home/aarav/Desktop/BIrd dataset/chara_dataset...
654158,Circus,aeruginosus,,birds,Western Marsh Harrier,Ulf Elman,Sweden,"Trögden, Uppsala län",59.5549,17.2787,...,yes,yes,no,,,no,,,48000,/home/aarav/Desktop/BIrd dataset/chara_dataset...
801401,Circus,aeruginosus,,birds,Western Marsh Harrier,Manceau Lionel,France,Arrondissement de Rochefort (near Marennes-Hi...,45.8382,-1.0476,...,no,no,no,,,no,Tascam DR-07X,Primo EM272,48000,/home/aarav/Desktop/BIrd dataset/chara_dataset...


Saving file  1078 / 1258 : https://xeno-canto.org/19727/download
Saving file  1079 / 1258 : https://xeno-canto.org/547426/download
Saving file  1080 / 1258 : https://xeno-canto.org/547787/download
Saving file  1081 / 1258 : https://xeno-canto.org/417796/download
Saving file  1082 / 1258 : https://xeno-canto.org/472784/download
Saving file  1083 / 1258 : https://xeno-canto.org/317228/download
Saving file  1084 / 1258 : https://xeno-canto.org/296965/download
Saving file  1085 / 1258 : https://xeno-canto.org/547626/download
Saving file  1086 / 1258 : https://xeno-canto.org/312272/download
Saving file  1087 / 1258 : https://xeno-canto.org/472783/download
Saving file  1088 / 1258 : https://xeno-canto.org/523180/download
Saving file  1089 / 1258 : https://xeno-canto.org/115161/download
Saving file  1090 / 1258 : https://xeno-canto.org/472782/download
Saving file  1091 / 1258 : https://xeno-canto.org/812239/download
Saving file  1092 / 1258 : https://xeno-canto.org/252906/download
Saving file

Saving file  1197 / 1258 : https://xeno-canto.org/468309/download
Saving file  1198 / 1258 : https://xeno-canto.org/758214/download
Saving file  1199 / 1258 : https://xeno-canto.org/638237/download
Saving file  1200 / 1258 : https://xeno-canto.org/553623/download
Saving file  1201 / 1258 : https://xeno-canto.org/564822/download
Saving file  1202 / 1258 : https://xeno-canto.org/142742/download
Saving file  1203 / 1258 : https://xeno-canto.org/499651/download
Saving file  1204 / 1258 : https://xeno-canto.org/844925/download
Saving file  1205 / 1258 : https://xeno-canto.org/716439/download
Saving file  1206 / 1258 : https://xeno-canto.org/528971/download
Saving file  1207 / 1258 : https://xeno-canto.org/781838/download
Saving file  1208 / 1258 : https://xeno-canto.org/460279/download
Saving file  1209 / 1258 : https://xeno-canto.org/701525/download
Saving file  1210 / 1258 : https://xeno-canto.org/547835/download
Saving file  1211 / 1258 : https://xeno-canto.org/471520/download
Saving fil

,gen,sp,ssp,group,en,rec,cnt,loc,lat,lng,...,bird-seen,animal-seen,playback-used,temp,regnr,auto,dvc,mic,smp,fullfilename
id,,,,,,,,,,,,,,,,,,,,,
19400,Eudynamys,scolopaceus,scolopaceus,birds,Asian Koel,Stuart Fisher,India,"Thiruvananthapuram, Kerala",8.4874,76.9520,...,unknown,unknown,unknown,,,no,,,44100,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
808437,Eudynamys,scolopaceus,,birds,Asian Koel,Zhou Zhe 周哲,China,"Shenzhen, Guangdong",22.5510,113.9971,...,no,no,no,,,no,,,48000,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
196584,Eudynamys,scolopaceus,,birds,Asian Koel,Frank Lambert,Indonesia,"Morotai, c km3 north of Sidowa, North Maluku",2.1402,128.5654,...,yes,yes,no,,,no,,,44100,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
547505,Eudynamys,scolopaceus,,birds,Asian Koel,Andrew Spencer,India,"Wildlife Institute of India Campus, Uttarakhand",30.2825,77.9735,...,yes,yes,no,,,no,,,48000,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
181991,Eudynamys,scolopaceus,,birds,Asian Koel,Marc Anderson,Sri Lanka,"Yala National Park, Buttuwa, Southern Province.",6.3161,81.4830,...,no,no,no,,,no,,,48000,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79068,Chrysomma,altirostre,griseigulare,birds,Jerdon's Babbler,Yong Ding Li,India,"Dibru-Saikhowa NP, Assam",27.6834,95.3473,...,unknown,unknown,unknown,,,no,,,44100,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
683645,Chrysomma,altirostre,,birds,Jerdon's Babbler,Thet Zaw Naing,Myanmar,"Nyaungdon, Maubin, Ayeyarwady",17.0496,95.6025,...,yes,yes,no,,,no,,,44100,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
21861,Chrysomma,altirostre,,birds,Jerdon's Babbler,Mathias Ritschard,India,"Dibru-Saikhowa NP, Assam",27.6834,95.3473,...,unknown,unknown,unknown,,,no,,,44100,/home/aarav/Desktop/BIrd dataset/chara/chara_d...
